## Pre Processing Steps

### Extract data from csv

In [3]:
import pandas as pd
df = pd.read_csv("merged_malware_features.csv")
df.info()
df.describe()
df.head()
df.isnull().sum().sort_values(ascending=False)

C:\Users\Hp\AppData\Local\Temp\ipykernel_8476\4268743908.py:2: DtypeWarning: Columns (285,287,289,291,293,295,297,299,301,303,305,307,309,311,313,315,317,319,321,323,325,327,329,331,333,335,337,339,341,343,345,347,349,351,353,355,357,359,361,363,365,367,369) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("merged_malware_features.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21125 entries, 0 to 21124
Columns: 371 entries, file_path to section_54_entropy
dtypes: float64(312), int64(2), object(57)
memory usage: 59.8+ MB


section_47_name       21124
section_46_entropy    21124
section_43_name       21124
section_45_entropy    21124
section_43_entropy    21124
                      ...  
byte_freq_12              0
byte_freq_11              0
byte_freq_10              0
file_path                 0
file_entropy              0
Length: 371, dtype: int64

In [4]:
df = pd.read_csv("merged_malware_features.csv", low_memory=False)

In [5]:
df = df.drop(columns=["file_path"], errors='ignore')


In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

# Select only section name columns (likely categorical)
section_name_cols = [col for col in df.columns if col.startswith("section_") and col.endswith("_name")]

# Fill NaNs and label encode
for col in section_name_cols:
    df[col] = df[col].fillna("unknown")
    df[col] = LabelEncoder().fit_transform(df[col])
    df[col] = df[col].astype(str).astype("category").cat.codes


In [7]:
df = df.fillna(df.median(numeric_only=True))

In [8]:
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].fillna("unknown")
    df[col] = df[col].astype(str).astype("category").cat.codes

In [9]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df)

# Train Isolation Forest
clf = IsolationForest(contamination=0.1, random_state=42)
# df['is_malware_predicted'] = clf.fit_predict(X_scaled)
df = pd.concat([df, pd.DataFrame({'is_malware_predicted': clf.fit_predict(X_scaled)})], axis=1)

# Convert output: -1 (anomaly) -> malware, 1 -> benign
df['is_malware_predicted'] = df['is_malware_predicted'].map({1: 0, -1: 1})

# View results
df[['is_malware_predicted']].value_counts()


is_malware_predicted
0                       19016
1                        2109
Name: count, dtype: int64

In [19]:
df

,file_size,entry_point,file_entropy,byte_freq_00,byte_freq_01,byte_freq_02,byte_freq_03,byte_freq_04,byte_freq_05,byte_freq_06,...,section_50_entropy,section_51_name,section_51_entropy,section_52_name,section_52_entropy,section_53_name,section_53_entropy,section_54_name,section_54_entropy,is_malware_predicted
0,192660,5040,7.238815,0.130250,0.003395,0.002356,0.003228,0.002237,0.001760,0.002092,...,7.08676,1,7.107906,1,7.060821,1,7.157307,1,7.067534,0
1,16496,6668,6.581416,0.239694,0.006729,0.004547,0.002728,0.006062,0.003637,0.001758,...,7.08676,1,7.107906,1,7.060821,1,7.157307,1,7.067534,0
2,196674,5040,7.148468,0.148032,0.003300,0.002298,0.003158,0.002191,0.001724,0.002064,...,7.08676,1,7.107906,1,7.060821,1,7.157307,1,7.067534,0
3,192653,5040,7.238893,0.130224,0.003395,0.002357,0.003229,0.002237,0.001760,0.002092,...,7.08676,1,7.107906,1,7.060821,1,7.157307,1,7.067534,0
4,256000,141450,6.509421,0.158457,0.003141,0.002445,0.001965,0.001832,0.001848,0.001824,...,7.08676,1,7.107906,1,7.060821,1,7.157307,1,7.067534,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21120,16128,6668,6.704042,0.219370,0.006944,0.004464,0.002852,0.006138,0.003658,0.001798,...,7.08676,1,7.107906,1,7.060821,1,7.157307,1,7.067534,0
21121,16000,6668,6.688090,0.221125,0.006812,0.005000,0.003000,0.006250,0.003687,0.002000,...,7.08676,1,7.107906,1,7.060821,1,7.157307,1,7.067534,0
21122,192538,5040,7.240497,0.129777,0.003376,0.002342,0.003199,0.002228,0.001740,0.002103,...,7.08676,1,7.107906,1,7.060821,1,7.157307,1,7.067534,0
21123,17120,6668,6.773117,0.213376,0.006834,0.004439,0.002804,0.005783,0.003680,0.001928,...,7.08676,1,7.107906,1,7.060821,1,7.157307,1,7.067534,0
